In [24]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\namra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
nlp = spacy.load("en_core_web_sm")

In [2]:
df = pd.read_excel(r'C:\Users\namra\Downloads\data-book.xlsx')

In [3]:
df1 = df[df['rating']==1]
df2 = df[df['rating']==2]
df3 = df[df['rating']==3]
df4 = df[df['rating']==4]
df5 = df[df['rating']==5]

In [4]:
text1 =''
for i in df1['body']:
    text1+=i
text2 =''
for i in df2['body']:
    text2+=i
text3 =''
for i in df3['body']:
    text3+=i
text4 =''
for i in df4['body']:
    text4+=i
text5 =''
for i in df5['body']:
    text5+=i

In [5]:
word_embeddings = {}
f = open(r'C:\Users\namra\glove.6B\glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [7]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [25]:
def extract(text1):
    doc = nlp(text1)
    tokens=[token.text for token in doc]
    #print(tokens)
    from string import punctuation
    punctuation=punctuation+'n'
    word_freq={}
    stop_words= list(STOP_WORDS)

    for word in doc:
        if word.text.lower() not in stop_words:
            if word.text.lower() not in punctuation:
                if word.text not in word_freq.keys():
                    word_freq[word.text]= 1
                else:
                    word_freq[word.text]+= 1 
        #print(word_freq)

    x=(word_freq.values())
    a=list(x)
    a.sort()
    max_freq=a[-1]

    for word in word_freq.keys():
        word_freq[word]=word_freq[word]/max_freq 
    #print(word_freq)

    sent_score={}
    sent_tokens=[sent for sent in doc.sents]
    #print(sent_tokens)

    for sent in sent_tokens:
        for word in sent:
            if word.text.lower() in word_freq.keys():
                if sent not in sent_score.keys():
                    sent_score[sent]=word_freq[word.text.lower()]
                else:
                    sent_score[sent]+= word_freq[word.text.lower()] 
    #print(sent_score)

    
    sentences = nlargest(n=1000,iterable=sent_score,key=sent_score.get) 
    #print(summary)

    for i in range(len(sentences)):
        sentences[i] = str(sentences[i])


    clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ", regex=True)
    clean_sentences = [s.lower() for s in clean_sentences]
    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

    sentence_vectors = []
    for i in clean_sentences:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100,))
        sentence_vectors.append(v)
    sim_mat = np.zeros([len(sentences), len(sentences)])

    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)

    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

    new_text1 = ''
    for i in range(20):
        new_text1 += ranked_sentences[i][1]
    return new_text1



In [9]:
import transformers
summarizer = transformers.pipeline("summarization")


No model was supplied, defaulted to t5-small (https://huggingface.co/t5-small)
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [26]:
new_text1 = extract(text1)
new_text2 = extract(text2)
new_text3 = extract(text3)
new_text4 = extract(text4)
new_text5 = extract(text5)

In [27]:


#summarizer2 = transformers.pipeline("summarization")
summarized1 = summarizer(new_text1, min_length=75, max_length=300)
summarized2 = summarizer(new_text2, min_length=75, max_length=300)
summarized3 = summarizer(new_text3, min_length=75, max_length=300)
summarized4 = summarizer(new_text4, min_length=75, max_length=300)
summarized5 = summarizer(new_text5, min_length=75, max_length=300)


In [19]:
summarized1

[{'summary_text': "Alberto Cairo's book is about visualization but only has three EXAMPLES . the book has a chart, which is re-used like 5 times in the book . it seems to be intended for people who have only used basic chart tools in excel and wish to produce only slightly less awful charts . but started reading and half the book is not there ."}]

In [20]:
summarized2

[{'summary_text': 'the book has a pivotal flaw: it assumes your data is already there, processed, understood, and ready to be presented; something that is by far the hardest to achieve .the book is not a stats book and should not be considered anything more than a graphics guide to power point and excel.the reader must belief that author opinion is good, and you are good to go.'}]

In [21]:
summarized3

[{'summary_text': "the first portion of the book is incorrectly cut and bound together at an angle . it's mostly a bunch of (very good) design types on presenting data . the problem with this book - it does not “show” you how to create the visuals . if you have it then it will probably come in handy here and there ."}]

In [22]:
summarized4

[{'summary_text': 'the author has done a great job explaining concepts on all the different attributes that make a good story when using data . this book is great for anyone who needs to improve the way they use and design charts, as well as those that need to present data to others . the book opens your eyes to some of the dumb ways we try to report out information to business leaders and also push us to think about the multiple ways we can present data.'}]

In [23]:
summarized5

[{'summary_text': "this book looks behind the scenes at how our brains work and how we process data . it gives us an incredible depth and breadth of insight into the mind and how it sees things . this book is one of the best data oriented books I've read in a long time . the author presents good ideas and examples, it helped me improve my visualization skills ."}]

In [28]:
print('summarized1',summarized1)
print('summarized2',summarized2)
print('summarized3',summarized3)
print('summarized4',summarized4)
print('summarized5',summarized5)

summarized1 [{'summary_text': "Alberto Cairo's book is about visualization but only has three EXAMPLES . the book has a chart, which is re-used like 5 times in the book . it seems to be intended for people who have only used basic chart tools in excel and wish to produce only slightly less awful charts . but started reading and half the book is not there ."}]
summarized2 [{'summary_text': 'the book has a pivotal flaw: it assumes your data is already there, processed, understood, and ready to be presented; something that is by far the hardest to achieve .the book is not a stats book and should not be considered anything more than a graphics guide to power point and excel.the reader must belief that author opinion is good, and you are good to go.'}]
summarized3 [{'summary_text': "the first portion of the book is incorrectly cut and bound together at an angle . it's mostly a bunch of (very good) design types on presenting data . the problem with this book - it does not “show” you how to c